In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 46.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 12.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 101.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 84.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
!pip install sentence-transformers

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from tqdm import tqdm

from nltk.corpus import stopwords
from string import punctuation

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Topic model
from bertopic import BERTopic
# Sentence model
from sentence_transformers import SentenceTransformer
# Dimension reduction
from umap import UMAP

from sklearn.utils.random import sample_without_replacement

In [ ]:
# sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2") # a bit faster, less precise
sentence_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2") # a bit slower, more precise

## Based on separate csv files for each channel

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/TG news channels/csv_files/data"
path_emb = "/content/drive/MyDrive/Colab Notebooks/TG news channels/csv_files/embeddings"
df_channels = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TG news channels/csv_files/channels.csv', index_col=0)
channels = df_channels['channel_name'].to_list()
cols_to_keep = ['channel_id', 'channel_name', 'id', 'date', 'cleaned_message', 'message', 'views']
missing_channels = []

for channel_name in channels:
  if os.path.exists(f"{path}/{channel_name}_messages.csv"):
    df = pd.read_csv(f"{path}/{channel_name}_messages.csv", index_col=0, usecols=cols_to_keep, low_memory=False)
    df = df[~df['cleaned_message'].isna()]
    df.reset_index(inplace=True)
    print(f"starting for {channel_name} ({df.shape[0]} messages)...")
    embeddings = sentence_model.encode(df['cleaned_message'], show_progress_bar=True)
    df = pd.concat([df, pd.DataFrame(embeddings)], axis=1)
    df.to_csv(f"{path_emb}/{channel_name}_embed.csv")
  else: missing_channels.append(channel_name)

starting for ntvnews (71640 messages)...


Batches:   0%|          | 0/2239 [00:00<?, ?it/s]

starting for bbcrussian (45503 messages)...


Batches:   0%|          | 0/1422 [00:00<?, ?it/s]

starting for news_1tv (21779 messages)...


...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

starting for breakingmash (454 messages)...


Batches:   0%|          | 0/15 [00:00<?, ?it/s]

starting for concordgroup_official (1018 messages)...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

In [ ]:
channels

['rt_russian',
 'ntvnews',
 'tvrussia1',
 'bbcrussian',
 'news_1tv',
 'redakciya_channel',
 'meduzalive',
 'mediazzzona',
 'thebell_io',
 'rian_ru',
 'readovkanews',
 'novaya_pishet',
 'rbc_news',
 'zvezdanews',
 'aifonline',
 'BFMnews',
 'fontankaspb',
 'forbesrussia',
 'gazetaru',
 'interfaxonline',
 'izvestia',
 'kommersant',
 'lentadnya',
 'lifenews',
 'mk_ru',
 'novaya_europe',
 'radiosvoboda',
 'rentv_news',
 'rgrunews',
 'riafan',
 'rusvesnasu',
 'svpressaru',
 'tass_agency',
 'truekpru',
 'uranews',
 'vedomosti',
 'vestiru24',
 'tsargradtv',
 'sashakots',
 'rybar',
 'voenacher',
 'vysokygovorit',
 'SolovievLive',
 'margaritasimonyan',
 'breakingmash',
 'tvc_ru',
 'strelkovii',
 'concordgroup_official']

In [ ]:
for channel_name in channels:
  if os.path.exists(f"{path}/{channel_name}_messages.csv") and not os.path.exists(f"{path_emb}/{channel_name}_embed.csv"):
    df = pd.read_csv(f"{path}/{channel_name}_messages.csv", index_col=0, usecols=cols_to_keep, low_memory=False)
    df = df[~df['cleaned_message'].isna()]
    df.reset_index(inplace=True)
    print(f"starting for {channel_name} ({df.shape[0]} messages)...")
    embeddings = sentence_model.encode(df['cleaned_message'], show_progress_bar=True)
    df = pd.concat([df, pd.DataFrame(embeddings)], axis=1)
    df.to_csv(f"{path_emb}/{channel_name}_embed.csv")

starting for tvrussia1 (15585 messages)...


Batches:   0%|          | 0/488 [00:00<?, ?it/s]

starting for zvezdanews (112705 messages)...


Batches:   0%|          | 0/3523 [00:00<?, ?it/s]

starting for uranews (71976 messages)...


...

Batches:   0%|          | 0/22 [00:00<?, ?it/s]

starting for vysokygovorit (145 messages)...


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

starting for tvc_ru (44350 messages)...


Batches:   0%|          | 0/1386 [00:00<?, ?it/s]

## Based on one csv file with texts

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/TG news channels/"
file = "msgs_dataset.csv"
chunk_size = 10**5
dataframes = []
batch = 0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):
    # check if the file already exists
    if not os.path.exists(f"{path}temp/news_w_embed_{batch}.csv"):
      print(f"Starting batch #{batch}...")
      # keep only relevant columns
      chunk = chunk[['channel_id', 'channel_name', 'msg_id', 'message'\
        , 'cleaned_message', 'date', 'views','number_replies', \
        'number_forwards', 'contains_media', 'media_type',\
        'has_url', 'url', 'domain', 'document_type']]
      # remove rows with NaN in cleaned_message
      chunk = chunk.dropna(subset=['cleaned_message'])
      # Calculate embeddings for each text in the chunk
      embeddings = sentence_model.encode(chunk['cleaned_message'], show_progress_bar=True)
      chunk.reset_index(inplace=True)
      chunk = pd.concat([chunk, pd.DataFrame(embeddings)], axis=1)
      chunk.to_csv(f"{path}temp/news_w_embed_{batch}.csv")
      print(f"Batch #{batch} saved.")
      # Append the chunk to the list of dataframes
      # dataframes.append(chunk)
    batch += 1

<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (35,37,41) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):
<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):
<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (14,16,17,18,19,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):
<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (14,16,17,18,19,26,27,28,29,35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):
<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (26,27,28,29,35,37) have mixed types. Specify 

Starting batch #29...


Batches:   0%|          | 0/2841 [00:00<?, ?it/s]

Batch #29 saved.


<ipython-input-10-7e916cc89bb2>:1: DtypeWarning: Columns (35,37) have mixed types. Specify dtype option on import or set low_memory=False.
  for chunk in pd.read_csv(path+file, index_col=0, chunksize=chunk_size):


Starting batch #30...


Batches:   0%|          | 0/695 [00:00<?, ?it/s]

Batch #30 saved.


# Encode summaries

## experiments - old

In [1]:
!pip install bertopic
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 91.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 102.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from tqdm import tqdm

from nltk.corpus import stopwords
from string import punctuation

import nltk
nltk.download('stopwords')

# Topic model
from bertopic import BERTopic
# Sentence model
from sentence_transformers import SentenceTransformer
# Dimension reduction
from umap import UMAP

from sklearn.utils.random import sample_without_replacement

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
default_model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2") # a bit slower, more precise
e5_model
'intfloat/multilingual-e5-base'
'intfloat/multilingual-e5-large'

In [3]:
from sentence_transformers import SentenceTransformer
sentences = ["Какая прекрасная погода"]

# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = SentenceTransformer('intfloat/multilingual-e5-base')
embeddings = model.encode(sentences)
print(embeddings)

[[-1.08991358e-02  3.04420423e-02 -3.86595889e-03  3.90509032e-02
   2.10677255e-02 -2.56722346e-02 -2.02704426e-02 -3.31883691e-02
   2.96315718e-02  2.74250843e-02 -3.94484550e-02  1.94231570e-02
   1.60815910e-01  2.47431640e-02 -3.62058096e-02 -3.57348919e-02
   2.12360378e-02 -1.76265035e-02  3.91042903e-02 -3.76279801e-02
   4.31610383e-02 -2.30338667e-02  4.33077216e-02 -2.98884753e-02
   1.42685985e-02 -1.17476312e-02  1.54987555e-02  1.55127179e-02
  -1.68872513e-02  2.11274475e-02  1.69034079e-02 -3.84450555e-02
   2.57360693e-02  2.20883992e-02  1.32094147e-02  8.27996507e-02
  -3.60354297e-02 -4.01341692e-02  2.49585453e-02  3.43339406e-02
   5.99696068e-03 -9.00180545e-03  2.47845636e-03 -9.38577950e-03
   3.73559408e-02  1.62491035e-02  6.03852496e-02  1.10255918e-02
  -1.51735656e-02 -1.70905888e-02  1.33687649e-02 -4.20379043e-02
   2.27475837e-02  5.69400052e-03 -6.01124205e-02 -1.41955400e-02
   1.31098274e-02  5.67114912e-02 -6.02778271e-02  3.85529213e-02
   9.65744

In [4]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ["Какая прекрасная погода"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings1 = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings1 = F.normalize(embeddings1, p=2, dim=1)

In [9]:
type(embeddings1), type(embeddings)

(torch.Tensor, numpy.ndarray)

In [18]:
 embeddings1 = embeddings1.detach().numpy()

In [34]:
diff = 0.0001
print(f"N of dims w/ difference of {diff} or less is {len(embeddings[embeddings1 - embeddings < diff])}")
print(f"Total N of dims - {len(embeddings1[-1])}")

N of dims w/ difference of 0.0001 or less is 768
Total N of dims - 768


In [20]:
np.corrcoef(embeddings1, embeddings)

array([[1., 1.],
       [1., 1.]])

In [23]:
np.dot(embeddings1, embeddings.T) / (np.linalg.norm(embeddings1) * np.linalg.norm(embeddings))

array([[1.]], dtype=float32)

Results are essentially the same. Can use pipeline method. Batching is also possible for pipeline

In [35]:
sentences = ["Какая прекрасная погода", "Какая прекрасная погода", "Какая прекрасная погода"]

# model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
model = SentenceTransformer('intfloat/multilingual-e5-base')

embeddings = model.encode(sentences)

In [56]:
import time

sentences = ["Какая прекрасная погода", "Какая прекрасная погода", "Какая прекрасная погода"]

start_time = time.time()
sentences = sentences*3000
embeddings = model.encode(sentences)

stop_time = time.time()

print(f"N: {len(sentences)}")
print(f"time: {stop_time - start_time}")

N: 9000
time: 6.749098062515259


In [54]:
model.device

device(type='cuda', index=0)

## Load libs & data

In [1]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 58.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=600b621cf5343c00747212a24a5a1132a214f5569f276a74f7390370c40311f1
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import numpy as np
import time

from sentence_transformers import SentenceTransformer

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TG news channels/filtered/df_ukr_20K_summaries.csv', index_col=0)

In [5]:
df.shape

(20359, 15)

2Do:
- e5 - raw texts, sum_news3k, sum_mbart
- default - sum_news3k, sum_mbart

# on real data

## raw, sum_news3k -> embed e5

In [6]:
model = SentenceTransformer('intfloat/multilingual-e5-large')
# e5_model
# 'intfloat/multilingual-e5-base'
# 'intfloat/multilingual-e5-large'

In [32]:
# text_cols = ['cleaned_message', 'summary_newspaper3k', 'summary_mbart']
text_cols = ['summary_newspaper3k', 'summary_mbart']

for col in text_cols:
   print(f"starting {col}..")
   texts = df[col].to_list()
   embeddings = model.encode(texts, show_progress_bar=True)
   df[f"{col}_emb_e5"] = [row.reshape(-1) for row in embeddings]

starting summary_newspaper3k..


Batches:   0%|          | 0/637 [00:00<?, ?it/s]

starting summary_mbart..


Batches:   0%|          | 0/637 [00:00<?, ?it/s]

In [33]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/TG news channels/filtered/df_ukr_20K_summaries.csv', index=False)

## sum_news3k, sum_mbart -> default embed

In [34]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

In [35]:
text_cols = ['summary_newspaper3k', 'summary_mbart']

for col in text_cols:
   print(f"starting {col}..")
   texts = df[col].to_list()
   embeddings = model.encode(texts, show_progress_bar=True)
   df[f"{col}_emb_mpnet"] = [row.reshape(-1) for row in embeddings]

starting summary_newspaper3k..


Batches:   0%|          | 0/637 [00:00<?, ?it/s]

starting summary_mbart..


Batches:   0%|          | 0/637 [00:00<?, ?it/s]

In [36]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/TG news channels/filtered/df_ukr_20K_summaries.csv', index=False)

In [40]:
df.columns

Index(['media_type_y', 'content_type', 'channel_id', 'channel_name', 'msg_id',
       'message', 'cleaned_message', 'date', 'views', 'ada_embedding',
       'summary_newspaper3k', 'summary_mbart', 'origin_len', 'news3k_len',
       'mbart_len', 'cleaned_message_emb_e5', 'summary_newspaper3k_emb_e5',
       'summary_mbart_emb_e5', 'summary_newspaper3k_emb_mpnet',
       'summary_mbart_emb_mpnet'],
      dtype='object')

In [39]:
len(df.summary_newspaper3k_emb_mpnet[0])

768

In [43]:
df.iloc[:, :-2].info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20359 entries, 0 to 20358
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   media_type_y                20359 non-null  object
 1   content_type                20359 non-null  object
 2   channel_id                  20359 non-null  int64 
 3   channel_name                20359 non-null  object
 4   msg_id                      20359 non-null  int64 
 5   message                     20359 non-null  object
 6   cleaned_message             20359 non-null  object
 7   date                        20359 non-null  object
 8   views                       20359 non-null  int64 
 9   ada_embedding               20359 non-null  object
 10  summary_newspaper3k         20359 non-null  object
 11  summary_mbart               20359 non-null  object
 12  origin_len                  20359 non-null  int64 
 13  news3k_len                  20359 non-null  in

In [45]:
df.to_pickle('/content/drive/MyDrive/Colab Notebooks/TG news channels/filtered/df_ukr_20K_summaries.pkl')